### Libraries

In [ ]:
!pip install pandas numpy scikit-learn

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'pandas'

### Data Loading

In [2]:
# Define file path
file_path = os.path.join("..", "..", "data-crunch-round-1", "train.csv")

# Load dataset
df = pd.read_csv(file_path)


NameError: name 'os' is not defined

### Pre Processing

In [ ]:
# Display basic info
display(df.head())
display(df.info())

# Convert 'Year', 'Month', 'Day' to datetime format
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df.sort_values(by=['Date']).reset_index(drop=True)

# Drop redundant columns (if 'Year', 'Month', 'Day' are no longer needed)
df.drop(columns=['Year', 'Month', 'Day'], inplace=True)

# Handle missing values (interpolation for continuous data, mode for categorical)
df.interpolate(method='linear', inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)

# Standardize temperature to °C (assuming Kelvin needs conversion)
def convert_temp(temp):
    return temp - 273.15 if temp > 100 else temp  # Assuming anything > 100K is Kelvin

df['Avg_Temperature'] = df['Avg_Temperature'].apply(convert_temp)
df['Avg_Feels_Like_Temperature'] = df['Avg_Feels_Like_Temperature'].apply(convert_temp)

# Encode categorical column 'Kingdom'
le = LabelEncoder()
df['Kingdom'] = le.fit_transform(df['Kingdom'])

# Handle outliers using IQR method for continuous columns
num_cols = ['Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction']
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.clip(df[col], lower_bound, upper_bound)

# Generate time-based features
df['Day_of_Year'] = df['Date'].dt.dayofyear
df['Week_of_Year'] = df['Date'].dt.isocalendar().week
df['Season'] = (df['Date'].dt.month % 12 + 3) // 3  # 1: Winter, 2: Spring, 3: Summer, 4: Fall

# Display processed dataset
display(df.head())